# AuraNet Training on Kaggle

Notebook này dùng để chạy training AuraNet trên môi trường Kaggle với hỗ trợ ghi log ra file để theo dõi quá trình training dễ dàng hơn.

# AuraNet Training on Kaggle

Notebook này dùng để chạy training AuraNet trên môi trường Kaggle.

## Thiết lập môi trường

Trước tiên, thiết lập các thư mục và sao chép các file cần thiết.

In [ ]:
import os
import sys
import shutil
import logging
from datetime import datetime

# Thiết lập các đường dẫn
WORKING_DIR = "/kaggle/working/AuraNet"
INPUT_DIR = "/kaggle/input"
AURANET_CODE_DIR = "/kaggle/input/auranet-code"  # Thay đổi tùy theo tên dataset của bạn
LOG_DIR = "/kaggle/working/AuraNet/logs"

# Tạo thư mục làm việc và logs
os.makedirs(WORKING_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

# Thêm đường dẫn vào Python path
sys.path.append(WORKING_DIR)
sys.path.append(os.path.join(WORKING_DIR, 'src'))

## Sao chép mã nguồn AuraNet

In [ ]:
# Sao chép tất cả file từ input dataset vào working directory
!cp -r {AURANET_CODE_DIR}/* {WORKING_DIR}/

# Kiểm tra file đã được sao chép
!ls -la {WORKING_DIR}

## Cài đặt thư viện cần thiết

In [ ]:
# Cài đặt các thư viện cần thiết
!pip install -r {WORKING_DIR}/requirements_kaggle.txt

## Cấu hình training

In [ ]:
# Thiết lập các tham số training
CONFIG_FILE = os.path.join(WORKING_DIR, "config_celeb_df_memory_optimized.yaml")
MODE = "pretrain"  # Options: pretrain, finetune, both
NUM_GPUS = 2
USE_PRETRAINED = "yes"
PRETRAINED_PATH = "/kaggle/input/convnextv2-pico/pytorch/default/1/convnextv2_pico_1k_224_fcmae.pt"

print(f"Training configuration:\n")
print(f"- Config file: {CONFIG_FILE}")
print(f"- Mode: {MODE}")
print(f"- GPUs: {NUM_GPUS}")
print(f"- Use pretrained: {USE_PRETRAINED}")
print(f"- Pretrained path: {PRETRAINED_PATH}")

## Kiểm tra GPU

In [ ]:
# Kiểm tra GPU có sẵn
!nvidia-smi

## Bắt đầu training

In [ ]:
# Log training start
logger.info(f"Starting training with mode: {MODE}, GPUs: {NUM_GPUS}")

# Chạy training với script tối ưu hóa (nếu có)
if os.path.exists(f"{WORKING_DIR}/train_optimized.py"):
    logger.info("Using optimized training script")
    !python {WORKING_DIR}/train_optimized.py \
        --config {CONFIG_FILE} \
        --mode {MODE} \
        --data_root {INPUT_DIR} \
        --gpus {NUM_GPUS} \
        --use_pretrained {USE_PRETRAINED} \
        --pretrained_path {PRETRAINED_PATH} \
        --memory_optimization \
        --enable_optimized_modules \
        --kaggle \
        --kaggle_working_dir {WORKING_DIR} 2>&1 | tee -a {log_file_path}
else:
    logger.info("Using standard training script")
    !python {WORKING_DIR}/train_celeb_df.py \
        --config {CONFIG_FILE} \
        --mode {MODE} \
        --data_root {INPUT_DIR} \
        --gpus {NUM_GPUS} \
        --use_pretrained {USE_PRETRAINED} \
        --pretrained_path {PRETRAINED_PATH} \
        --kaggle \
        --kaggle_working_dir {WORKING_DIR} 2>&1 | tee -a {log_file_path}
        
logger.info("Training command completed")

## Kiểm tra kết quả

In [ ]:
# Kiểm tra thư mục logs
LOGS_DIR = os.path.join(WORKING_DIR, "logs")
!ls -la {LOGS_DIR}/celeb_df/

## Lưu checkpoints

Đảm bảo lưu checkpoints ra ngoài để có thể tiếp tục training sau này.

In [ ]:
# Nén thư mục logs để tải xuống
!tar -czvf /kaggle/working/auranet_logs.tar.gz {LOGS_DIR}

print("Logs compressed and ready for download.")

In [ ]:
# Thiết lập logging
def setup_logging(log_dir=LOG_DIR, level=logging.INFO):
    """Set up logging to file and console"""
    # Create timestamp for filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_file = f"auranet_kaggle_{timestamp}.log"
    log_path = os.path.join(log_dir, log_file)
    
    # Configure root logger
    logger = logging.getLogger()
    logger.setLevel(level)
    
    # Remove existing handlers
    for handler in logger.handlers[:]:
        logger.removeHandler(handler)
    
    # Create console handler
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setLevel(level)
    console_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    console_handler.setFormatter(console_format)
    logger.addHandler(console_handler)
    
    # Create file handler
    file_handler = logging.FileHandler(log_path)
    file_handler.setLevel(level)
    file_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(file_format)
    logger.addHandler(file_handler)
    
    print(f"Logging initialized. Log file: {log_path}")
    logger.info("AuraNet Kaggle training started")
    
    return logger, log_path

# Set up logging
logger, log_file_path = setup_logging()
logger.info("Environment setup completed")
print(f"All logs will be saved to: {log_file_path}")